In [1]:
import sklearn
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("/home/dev/Hate_speech_detection/dataset.csv", encoding = 'latin', header=None)


In [3]:
df.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [4]:
df = df.rename(columns={0: 'target', 1: 'id', 2: 'date', 3: 'query', 4: 'username', 5: 'content'})

In [21]:
df.columns

Index(['target', 'id', 'date', 'query', 'username', 'content'], dtype='object')

In [10]:
def del_column(df, cols):
    for col in cols:
        del df[col]
    return df    

In [16]:
sum(df["target"] == 4)

800000

In [9]:
df.isnull().sum()

target      0
id          0
date        0
query       0
username    0
content     0
dtype: int64

In [10]:
# remove special characters using the regular expression library
import re

#set up punctuations we want to be replaced
REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\|)|(\()|(\))|(\[)|(\])|(\%)|(\$)|(\>)|(\<)|(\{)|(\})")
REPLACE_WITH_SPACE = re.compile("(<br\s/><br\s/?)|(-)|(/)|(:).")

In [11]:
import preprocessor as p

# custum function to clean the dataset (combining tweet_preprocessor and reguar expression)
def clean_tweets(df):
  tempArr = []
  for line in df:
    # send to tweet_processor
    tmpL = p.clean(line)
    # remove puctuation
    tmpL = REPLACE_NO_SPACE.sub("", tmpL.lower()) # convert all tweets to lower cases
    tmpL = REPLACE_WITH_SPACE.sub(" ", tmpL)
    tempArr.append(tmpL)
  return tempArr

In [12]:
# clean training data
train_tweet = clean_tweets(df['content'])
train_tweet = pd.DataFrame(train_tweet)
train_tweet.head()

,0
0,awww thats a bummer you shoulda got david ca...
1,is upset that he cant update his facebook by t...
2,i dived many times for the ball managed to sav...
3,my whole body feels itchy and like its on fire
4,no its not behaving at all im mad why am i her...


In [13]:
df.drop(['id','date','query','username'], axis=1, inplace=True)


In [14]:
df["clean_tweet"] = train_tweet
df['target'] = df['target'].replace([0, 4],['Negative','Positive'])

df.head(20)

,target,content,clean_tweet
0,Negative,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",awww thats a bummer you shoulda got david ca...
1,Negative,is upset that he can't update his Facebook by ...,is upset that he cant update his facebook by t...
2,Negative,@Kenichan I dived many times for the ball. Man...,i dived many times for the ball managed to sav...
3,Negative,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire
4,Negative,"@nationwideclass no, it's not behaving at all....",no its not behaving at all im mad why am i her...
5,Negative,@Kwesidei not the whole crew,not the whole crew
6,Negative,Need a hug,need a hug
7,Negative,@LOLTrish hey long time no see! Yes.. Rains a...,hey long time no see yes rains a bit only a bi...
8,Negative,@Tatiana_K nope they didn't have it,nope they didnt have it
9,Negative,@twittera que me muera ?,que me muera


In [15]:
from sklearn.model_selection import train_test_split

# extract the labels from the train data
y = df.target.values

# use 70% for the training and 30% for the test
x_train, x_test, y_train, y_test = train_test_split(df.clean_tweet.values, y, 
                                                    stratify=y, 
                                                    random_state=1, 
                                                    test_size=0.3, shuffle=True)

                                                    

In [16]:
print('Train dataset shape: {}'.format(x_train.shape))
print('Test dataset shape: {}'.format(x_test.shape))

Train dataset shape: (1120000,)
Test dataset shape: (480000,)


In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [18]:
vect = TfidfVectorizer(stop_words='english', max_df=0.7)
x_train_vec = vect.fit_transform(x_train)
x_test_vec = vect.transform(x_test)

In [19]:
from sklearn.linear_model import PassiveAggressiveClassifier


In [20]:
classifier = PassiveAggressiveClassifier(max_iter=50)
classifier.fit(x_train_vec, y_train)


PassiveAggressiveClassifier(max_iter=50)

In [21]:
from sklearn.metrics import accuracy_score
y_pred = classifier.predict(x_test_vec)
score = accuracy_score(y_test, y_pred)
print("Accuracy score for SVC is: ", score * 100, '%')

Accuracy score for SVC is:  73.13916666666667 %


In [22]:
def func(stri):
    input_data = [stri]
    vectorize_input_data = vect.transform(input_data)
    pred = classifier.predict(vectorize_input_data)
    print(pred)

In [23]:
func('i will kidnapp you')

['Negative']


In [31]:
import pickle

In [32]:
pickle1 = {
    "vect":vect,
    "classifier":classifier
}
with open("model_pickle_hate", "wb") as f:
    pickle.dump(pickle1, f)

In [33]:
with open("model_pickle_hate", "rb") as f:
    mp = pickle.load(f)
    classifier = mp["classifier"]

In [34]:
stri = "i kill you"
input_data = [stri]
vectorize_input_data = vect.transform(input_data)
classifier.predict(vectorize_input_data)

array(['Negative'], dtype='<U8')